In [20]:
# Instalação das bibliotecas (execute apenas se não estiverem instaladas)
# !pip install gensim matplotlib scikit-learn pandas numpy spacy plotly

# Importações básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import gensim.downloader as api
from sklearn.manifold import TSNE
import re
import nltk
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'numpy.polynomial'

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

In [ ]:
# Dados de exemplo - críticas de filmes (simplificadas)
textos = [
    "Este filme é incrível, adorei a atuação do protagonista",
    "A direção de fotografia é espetacular e o roteiro é envolvente",
    "Péssimo filme, desperdicei meu tempo assistindo isso",
    "Os atores são talentosos mas o roteiro é fraco",
    "Cinematografia belíssima, recomendo assistir no cinema",
    "Não gostei da história, personagens mal desenvolvidos",
    "A trilha sonora combina perfeitamente com as cenas",
    "Filme entediante, previsível do início ao fim",
    "Os efeitos especiais são impressionantes, tecnologia de ponta",
    "História emocionante, chorei no final do filme"
]

# Verificando os dados
for i, texto in enumerate(textos[:3]):  # Mostrando apenas os 3 primeiros
    print(f"Texto {i+1}: {texto}")

In [ ]:
from nltk.corpus import stopwords

def preprocessar_texto(texto):
    # Converter para minúsculas
    texto = texto.lower()

    # Remover caracteres especiais e números
    texto = re.sub(r'[^a-záàâãéèêíïóôõöúçñ ]', '', texto)

    # Tokenizar
    tokens = word_tokenize(texto)

    # Remover stopwords (opcional, dependendo da aplicação)
    stop_words = set(stopwords.words('portuguese'))
    tokens = [token for token in tokens if token not in stop_words]

    return tokens

# Aplicar pré-processamento a todos os textos
textos_preprocessados = [preprocessar_texto(texto) for texto in textos]

# Verificar resultado
print("Exemplo de texto original:")
print(textos[0])
print("\nDepois do pré-processamento:")
print(textos_preprocessados[0])

In [ ]:
# Definir parâmetros do modelo
vector_size = 100    # Dimensionalidade dos vetores
window = 5           # Tamanho da janela de contexto
min_count = 1        # Frequência mínima das palavras
workers = 4          # Número de threads para treinamento
sg = 1               # Modelo Skip-gram (1) ou CBOW (0)

In [ ]:
# Treinar o modelo
model = Word2Vec(
    sentences=textos_preprocessados,
    vector_size=vector_size,
    window=window,
    min_count=min_count,
    workers=workers,
    sg=sg
)

print(f"Modelo treinado com {len(model.wv.key_to_index)} palavras no vocabulário")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

# Dados rotulados para exemplo
textos_rotulados = textos  # Usando os mesmos textos de antes
sentimentos = [1, 1, 0, 0, 1, 0, 1, 0, 1, 1]  # 1: positivo, 0: negativo

# Função para gerar vetores de documento usando embeddings
def texto_para_vetor(texto, modelo):
    """Converte um texto em um vetor médio dos embeddings de suas palavras"""
    palavras = preprocessar_texto(texto)
    # Filtrar palavras que estão no vocabulário do modelo
    palavras_no_vocab = [p for p in palavras if p in modelo.wv]
    if not palavras_no_vocab:
        # Se nenhuma palavra estiver no vocabulário, retorna vetor de zeros
        return np.zeros(modelo.vector_size)
    # Calcular a média dos vetores das palavras
    vetores = [modelo.wv[palavra] for palavra in palavras_no_vocab]
    return np.mean(vetores, axis=0)

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    textos_rotulados, sentimentos, test_size=0.3, random_state=42
)

# 1. Abordagem com TF-IDF
vectorizer = TfidfVectorizer(max_features=100)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

clf_tfidf = LogisticRegression(random_state=42)
clf_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)

# 2. Abordagem com Word Embeddings
X_train_emb = np.array([texto_para_vetor(texto, model) for texto in X_train])
X_test_emb = np.array([texto_para_vetor(texto, model) for texto in X_test])

clf_emb = LogisticRegression(random_state=42)
clf_emb.fit(X_train_emb, y_train)
y_pred_emb = clf_emb.predict(X_test_emb)

# Comparar resultados
print("\nResultados com TF-IDF:")
print(classification_report(y_test, y_pred_tfidf))

print("\nResultados com Word Embeddings:")
print(classification_report(y_test, y_pred_emb))
